# Chapter 7: Functions II: Recursion

In [1]:
from random import randint

Functions are a way to encapsulate parts of a program.

Here, you'll put together everything we've learned so far.

First, define a function that calculates the next number in the Collatz chain.

In [2]:
def nextCollatzNumber(initialNum):
    if (initialNum % 2 == 0):
        return initialNum // 2
    else:
        return (initialNum * 3) + 1

We can now generate the collatz path for 9.

In [3]:
def collatz9():
    n = 9
    pathLength = 0
    print(pathLength , n)
    while (n != 1):
        n = nextCollatzNumber(n)
        pathLength = pathLength + 1
        print(pathLength, n)

We could also make that loop a function itself.

This function will calculate the length of the collatz path of *n*.

In [4]:
def collatzPath(n):
    if(n < 1): #A little safety feature: If it's called with n<1,
               #safely return without hanging forever. 
        return -1
    pathLength = 0
    while (n != 1):
        n = nextCollatzNumber(n)
        pathLength = pathLength + 1
    return pathLength

Give it a whirl!

In [5]:
print("The path length of 10 is ",collatzPath(10))

The path length of 10 is  6


Say I want to find the largest number *randint*(1,10000)
returns when I run it *n* times.

In [ ]:
def largestRand(n):
    largest = 0
    for i in range(n):
        a = randint(1,10000)
        if(a > largest):
            largest = a
    return largest

Give this a whirl, too!

In [ ]:
print("after 100 passes, the largest number returned was ",largestRand(100))

Fundamentally, functions provide a way to encapsulate one logical chunk of
a program. 

Consider a program to perform [monte carlo integration](http://en.wikipedia.org/wiki/Monte Carlo), where we're sampling a function a bunch of times, and the area under that function is its average value times the width of the interval.)

There are essentially three components to such a program:
1. The main controller that operates the other bits and displays output
2. The integrator that samples the function and determines the area under it.
3. The function being sampled.

On the one hand, we can implement it as one program.

This program takes as arguments the number of samples, the function being
integrated, and the bounds of the integration.

In [6]:
def monteCarloSingle(numSamples, fname, start, stop):
    area=0
    for i in range(numSamples):
        #Get a random number between the start and stop range:
        randomNum = randint(0,1000000) #get a random number
        #and convert that integer to a number in our sample domain
        x = randomNum/1000000 * (start-stop) + start
        if(fname == 's'): #calculate the sine of the number with a taylor series
            funVal = x - 1/6*x**3 + 1/120*x**5 - 1/5040*x**7 + 1/362880*x**9
        if(fname == 'c'): #calculate the cosine.
            funVal = 1-1/2*x**2 + 1/24*x**4 - 1/720*x**6+1/40320*x**8
        if(fname == 'e'): #calculate e^x.
            funVal = 1+x+1/2*x**2 + 1/6*x**3 + 1/24*x**4 + 1/120*x**5+1/720*x**6+1/5040*x**7
        area = area + funVal/numSamples*(stop-start)
    print("the area between ",start," and ", stop, " is ",area)

Okay, now I can run it:

In [7]:
monteCarloSingle(10000, 's', 0, 1)

the area between  0  and  1  is  -0.4524011188786845


Uh-oh.

It's negative. 

There must be a bug somewhere in the code.

But since the program isn't decomposed, I can't test the individual bits;
I have to analyze the whole thing at once.

The problem may be in the Taylor series, it may be in the accumulation of the area, it could be anywhere.

Now, we can decompose the program using functions:

In [24]:
#First, the main controller:
def monteCarloDecomposed(numSamples, fname, start, stop):
    area = mcIntegrate(numSamples, fname, start, stop)
    print("the area between ",start," and ", stop, " is ",area)

#Now, the integrator:
def mcIntegrate(numSamples, fname, start, stop):
    area = 0
    for i in range(numSamples):
        #First, get a random number in the desired range.
        x = randomInRange(start, stop)
        #then evaluate the function at that location
        funVal = evaluateFunction(fname, x)
        area = area + funVal/numSamples*(stop-start)
    return area



Okay, so now I need to write *randomInRange* and *evaluateFunction*.

In [25]:
def randomInRange(start, stop):
    randomNum = randint(0,1000000)
    retVal = randomNum/1000000 * (start-stop) + start
    return retVal

def evaluateFunction(fname, x):
    if(fname == 's'): 
        funVal = x - 1/6*x**3 + 1/120*x**5 - 1/5040*x**7 + 1/362880*x**9
    if(fname == 'c'): 
        funVal = 1-1/2*x**2 + 1/24*x**4 - 1/720*x**6+1/40320*x**8
    if(fname == 'e'): 
        funVal = 1+x+1/2*x**2 + 1/6*x**3 + 1/24*x**4 + 1/120*x**5+1/720*x**6+1/5040*x**7
    return funVal

In [26]:
monteCarloDecomposed(10000, 's', 0,1)

the area between  0  and  1  is  -0.45958562685024124


Okay, so the bug is still there.

But I can play with the components to seewhich one has a bug. I think it's the Taylor expansion. sin(0.5) = 0.479, so

In [27]:
evaluateFunction('s', 0.5)

0.4794255386164159

Okay, so the bug doesn't seem to be in *evaluateFunction*.

Perhaps it's in *randomInRange*?

In [28]:
randomInRange(0,1)

-0.874639

A-ha! I have now narrowed down the bug to one line of code. 

Now: *Recursion*.

This is a case where a function calls itself. 

Let me define the factorial thus:
*factorial*(1) = 1

*factorial*(*n*) = *n* x *factorial*(*n*-1)

So, for example, *factorial*(5) = 5 x *factorial*(4)

Here's the *factorial* function, defined recursively.

In [29]:
def factRec(n):
    if (n <= 1):
        return 1
    else:
        return n * factRec(n-1)

Why did I use the case *n*<=1, rather than *n* == 1? 

Consider the case where I call *factRec*(0). 

It never stops, because *n* is never going to be 1.

So it's a safeguard.

Another example:

Computing the summation of 1/*i* for *i* = 1 to *N*

In [30]:
def sumInverse(n):
    return 1 if n <=1 else 1/n + sumInverse(n-1)

Here's another example: the Ackermann function. 

It's an interesting function that is designed to generate VERY large numbers.

*A*(0,*n*) = *n*+1

*A*(*m*,0) = *A*(*m*-1,1)

*A*(*m*,*n*) = A(*m*-1,*A*(*m*,*n*-1))

Do NOT call this with arguments larger than 3. The time it takes grows faster
than the numbers it produces. 

In [31]:
def ackermann(m,n):
    if(m == 0):
        return n+1
    if(n == 0):
        return ackermann(m-1,1)
    return ackermann(m-1, ackermann(m,n-1))

####   *Exercises*

1 - Of the numbers 1 to 95 (including 95), which has the largest Collatz path?

2 - Fix the decomposed Monte-carlo integrator. 

3 - A ball is dropped from an initial height Ho. It bounces to 75% of its drop
height on each bounce.

How many bounces will it take for the bounce to be less than 1 mm if the ball is dropped from 2m?

First, describe the problem recursively. Second, implement your solution, using only recursion. 

4 - Write a function that prints the sequence of moves needed to solve the Tower of Hanoi puzzle for *n* disks.

In [22]:
def printMoves(ndisks):
    hanoi("L", "R", "M", ndisks)

#Example:
#>>> printMoves(3)
#L R
#L M
#R M
#L R
#M L
#M R
#L R


5 - Given two strings of dna,

*a*. Make sure that both strings are valid DNA.

*b*. Make sure they have the same length.

*c*. Calculate the number of letters that are different between them.

The function should return the number of differences, or -1 if there was a problem with *a* or *b*.

(Incidentally, this metric is called the *Hamming Weight*, and is used in both bioinformatics and spell-checkers.)

You may use either recursion or iteration, but if you find this topic interesting, I should warn you that all of the more advanced algorithms are inherently recursive (there is no direct iterative equivalent), so now's a great time to practice.